In [1]:
import importlib
import e7awgsw
import bokeh.io
import bokeh.plotting
import bokeh.models
import qubelsi.qube

try:
    qubeui
except NameError:
    import qubeui
else:
    importlib.reload(qubeui)

bokeh.plotting.output_notebook()

qube = qubelsi.qube.Qube(
    '10.5.0.23', # IP address of eXtickGE
    './adi_api_mod', # Path to API
)
qubeui.StartupUI(qube)
#startup.

Loading BokehJS ...

Text(value='10.5.0.23', description='IP Address for eXtickGE', disabled=True, style=DescriptionStyle(descripti…

Text(value='/home/sio3/QuBE_Char/adi_api_mod', description='Path to API', disabled=True, layout=Layout(width='…

Button(description='Do init', layout=Layout(height='80px', width='50%'), style=ButtonStyle())

Button(description='Restart AD9082', layout=Layout(height='80px', width='50%'), style=ButtonStyle())

In [2]:
try:
    instruments
except NameError:
    import instruments
else:
    importlib.reload(instruments)
    
visa = instruments.new_resource_manager()
ms2038 = instruments.MS2038_SPA(visa)
#ms2038.query('*IDN?')

In [4]:
r = ms2038.measure()

In [5]:
a = r['numpy_array']
p = bokeh.plotting.figure(toolbar_location=None)
p.line(a[0],a[1])
bokeh.plotting.show(p)

In [32]:
importlib.reload(qubeui)

p = qubeui.AwgPanel(qube)
p.display()

In [1]:
fr = 8500
fl = 11700
fi = fl - fr
set_flo_fnco_according_to_frf(port[8], fr, fi) # MHz
set_vatt(port[8].upconv.vatt, 0xA00)

NameError: name 'set_flo_fnco_according_to_frf' is not defined

In [85]:
LOR = 121
LOC = 100
LOP = 110

set_lmx2594_freq_100M(qube.lmx2594[0], LOR) # 0 LO for READOUT/READIN1
set_lmx2594_freq_100M(qube.lmx2594[1], LOP) # 2 LO for PUMP1
set_lmx2594_freq_100M(qube.lmx2594[2], LOC) # 5 LO for CTRL1
set_lmx2594_freq_100M(qube.lmx2594[3], LOC) # 6 LO for CTRL2
set_lmx2594_freq_100M(qube.lmx2594[4], LOC) # 7 LO for CTRL3
set_lmx2594_freq_100M(qube.lmx2594[5], LOC) # 8 LO for CTRL4 <-
set_lmx2594_freq_100M(qube.lmx2594[6], LOP) # 11 LO for PUMP2
set_lmx2594_freq_100M(qube.lmx2594[7], LOC) # 13 LO for READOUT/READIN2
for o in qube.lmx2594:
    apply_lmx2594(o)

In [86]:
FR = 1656.25
FC = 1578.125
FP = 1656.25
MHz = 1e+6
qube.ad9082[0].set_nco(ch=0, freq=FR*MHz) # 0 READOUT1
qube.ad9082[0].set_nco(ch=1, freq=FP*MHz) # 2 PUMP1
qube.ad9082[0].set_nco(ch=2, freq=FC*MHz) # 5 CTRL1
qube.ad9082[0].set_nco(ch=3, freq=FC*MHz) # 6 CTRL2
qube.ad9082[1].set_nco(ch=0, freq=FC*MHz) # 7 CTRL3
qube.ad9082[1].set_nco(ch=1, freq=FC*MHz) # 8 CTRL4
qube.ad9082[1].set_nco(ch=2, freq=FP*MHz) # 11 PUMP2
qube.ad9082[1].set_nco(ch=3, freq=FR*MHz) # 13 READOUT2

'set_nco: freq=1656250000, ch=3, COARSE_MODE, DAC_MODE\nadi_ad9081_device_api_revision_get(...)\nadi_ad9081_device_init(...)\napi v1.2.0 commit e1ac8d2 for ad9081 \nhost is using little endian mode.\nadi_ad9081_device_spi_config(...)\nadi_ad9081_device_reg8_access_check(...)\nadi_ad9081_device_power_status_check(...)\nadi_ad9081_device_clk_config_set(...)\nadi_ad9081_device_boot_pre_clock(...)\nadi_ad9081_device_chip_id_get(...)\ndevice is ad9082 r3\nadi_ad9081_device_laminate_id_get(...)\nadi_ad9081_device_die_id_get(...)\nadi_ad9081_dac_digital_logic_enable_set(...)\nadi_ad9081_device_aclk_receiver_enable_set(...)\nadi_ad9081_device_clk_pll_enable_set(...)\nadi_ad9081_adc_clk_div_set(...)\nadi_ad9081_device_clk_up_div_set(...)\nadi_ad9081_device_boot_post_clock(...)\nadi_ad9081_adc_clk_enable_set(...)\nadi_ad9081_dac_duc_nco_set(...)\nadi_ad9081_dac_duc_nco_enable_set(...)\nadi_ad9081_dac_select_set(...)\nadi_ad9081_hal_calc_tx_nco_ftw(...)\nadi_ad9081_dac_duc_nco_ftw_set(...)\nadi_a

In [118]:
def read_peak(inst, fc): # MHz
    #inst.write(':INIT:CONT OFF; *WAI')
    inst.write(':FREQ:CENT {} MHZ; *WAI'.format(fc))
    #inst.write(':INIT ONCE')
    time.sleep(2)
    inst.write('CALC:MARK1:MAX')
    x = inst.query('CALC:MARK1:X?')
    y = inst.query('CALC:MARK1:Y?')
    #inst.write(':INIT:CONT ON; *WAI')
    return x, y


In [114]:
fr, fi, fr + fi

(8500, 3500, 12000)

In [90]:
offset = lambda l, d: [v + i*d for i, v in enumerate(l)]

def plot_spurious_multi(p, d):
    p.renderers = []
    p.multi_line(offset(d['freq'],10), offset(d['trace'],0))
    bokeh.plotting.show(p)    
    
def plot_spurious(p, d, i):
    p.renderers = []
    p.line(d['freq'][i], d['trace'][i])
    bokeh.plotting.show(p)    

In [140]:
import pyvisa
import time
m = pyvisa.ResourceManager('@py')
inst = m.open_resource("TCPIP0::10.250.0.10::inst0::INSTR")
inst.query('*IDN?')

# MS2038Cから測定データを読み出す
def get_freq_from_ms2038c(inst):
    myfind = lambda k, l: next(filter(lambda x: x.startswith("{}=".format(k)), l))
    myconv = lambda s: float(s.split('=')[1].replace(' MHZ', ''))
    inst.timeout = 10000
    preamble = inst.query(':TRAC:PRE?')[10:].split(',')
    fstart = myconv(myfind('START_FREQ', preamble))
    fstop = myconv(myfind('STOP_FREQ', preamble))
    npts = int(float(myfind('UI_DATA_POINTS', preamble).split('=')[1]))
    return np.linspace(fstart, fstop, npts)

def get_trace_from_ms2038c(inst):
    return np.loadtxt(inst.query(':TRAC?')[6:].split(','))

def sweep_ms2038c(inst):
    inst.write(':INIT')
    while True:
        time.sleep(.1)
        if inst.query(':STAT:OPER?') == '256':
            break
    return True


In [129]:
import numpy as np
from IPython.display import clear_output

# ADRF6780のVATTを変更しながらスペクトルを取得する
set_flo_fnco_according_to_frf(port[8], 8600, 1800)

def sweep_ms2038c(inst):
    inst.write(':INIT')
    while True:
        time.sleep(.1)
        if inst.query(':STAT:OPER?') == '256':
            break
    return True

inst.write(':INIT:CONT OFF')
freq = get_freq_from_ms2038c(inst)
data = {'freq':[], 'trace':[], 'vatt':[]}

p = bokeh.plotting.figure(height=300, toolbar_location=None, sizing_mode="stretch_width")
bokeh.plotting.output_notebook()
plot_spurious_multi(p, data)
for i, v in enumerate(range(0x500,0x700,0x100)):
    set_vatt(port[8].upconv.vatt, v)
    sweep_ms2038c(inst)
    data['vatt'].append(hex(v))
    data['freq'].append(freq)
    data['trace'].append(get_trace_from_ms2038c(inst))
    clear_output()
    plot_spurious_multi(p, data)

In [139]:
# ADRF6780のVATTを変更しながらスペクトルを取得する
import pickle

inst.write(':INIT:CONT OFF')
freq = get_freq_from_ms2038c(inst)

FL = 15000
#for FL in range(10400, 11001, 200):
# FI = 3200
for FI in range(1800,3201,10):
    FR = FL - FI
    fname = 'if{}rf{}.pickle'.format(FI, FR)
    set_flo_fnco_according_to_frf(port[8], FR, FI)
    data = {'freq':[], 'trace':[], 'vatt':[]}
    p = bokeh.plotting.figure(height=300, toolbar_location=None, sizing_mode="stretch_width")
    #for i, v in enumerate(range(0xD00,0x200,-0x100)):
    for i, v in enumerate(range(0xA00,0x900,-0x100)):
        set_vatt(port[8].upconv.vatt, v)
        sweep_ms2038c(inst)
        data['vatt'].append(hex(v))
        data['freq'].append(freq)
        data['trace'].append(get_trace_from_ms2038c(inst))
        clear_output()
        plot_spurious_multi(p, data)    
    with open(fname, 'wb') as f:
        pickle.dump(data, f)


KeyboardInterrupt: 